# Genotype QC UKBB data

## Aim

Generate a set of Qc'ed genotype array data for the whole dataset of the UKBB 

## Output file

Final bed file with all the QC steps located here:

`~/UKBiobank/genotype_files_processed/083021_sample_variant_qc_final/cache/UKB_genotypedatadownloaded083019.083021_sample_variant_qc_final.filtered.extracted.bed`

# Variant QC summary

Original file downloaded from the UKBB 

~/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.bed

- Starting number of variants: 784,256 (starting with only autosomal variants)
- Starting number of individuals: 488,377 (individuals that have both genotype and phenotype information)

* Autosomal variants --> 784,256
* Covered by both arrays (array : (0/1/2) Presence of SNP on genotyping arrays 0=BiLEVE, 1=Axiom, 2=both) --> 733,322
* Batch level QC (BATCH_qc : (0/1) For each batch (Batch_b001-b095,UKBiLEVEAX_b1-b11), SNP passed all QC tests (no/yes)) --> 687,004
* SNPs only (remove indels) --> 674,489

In [11]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()



In [12]:
setwd('~/UKBiobank/genotype_files_processed')

## Step 1. Import file needed (ukb_snp_qc.txt)

In [34]:
ukb_snp_qc <- read.table('~/UKBiobank/data/genotype_files/ukb_snp_qc.txt', header=TRUE, sep=" ")

In [35]:
# variants genotyped and thus listed in this dataframe = 805,426
head(ukb_snp_qc)
nrow(ukb_snp_qc)

,rs_id,affymetrix_snp_id,affymetrix_probeset_id,chromosome,position,allele1_ref,allele2_alt,strand,array,Batch_b001_qc,...,PC32_loading,PC33_loading,PC34_loading,PC35_loading,PC36_loading,PC37_loading,PC38_loading,PC9_loading.3,PC40_loading,in_Phasing_Input
,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,rs28659788,Affx-13546538,AX-32115783,1,723307,C,G,+,0,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
2,rs116587930,Affx-35298040,AX-37361813,1,727841,G,A,+,2,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
3,rs116720794,Affx-13637449,AX-32137419,1,729632,C,T,+,2,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
4,rs3131972,Affx-13945728,AX-13191280,1,752721,A,G,+,2,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
5,rs12184325,Affx-13963217,AX-11194291,1,754105,C,T,+,2,1,...,-0.00347144,0.00589896,-0.00373881,-0.00189571,-0.00286888,0.000792278,-0.00191024,0.00307453,-0.000410934,1
6,rs3131962,Affx-13995532,AX-32225497,1,756604,A,G,+,2,1,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,1


[1] 805426

## Step 2. Select autosomal variants only  

In [37]:
# subset dataframe 

data.SNP_QC <-subset(ukb_snp_qc, chromosome %in% 1:22)

# autosomal variants listed in dataframe = 784,256
nrow(data.SNP_QC)

[1] 784256

In [ ]:
# create list.nonautosomalvariants for reference

list.nonautosomalvariants <-subset(ukb_snp_qc, !chromosome %in% 1:22, select=c(rs_id))
outfile.name1 <- "list.nonautosomalvariants.txt"
write.table(list.nonautosomalvariants, outfile.name1, quote=FALSE, col.names=TRUE, row.names=FALSE, sep="\t")

## Step 3. Select variants typed in both arrays

In [38]:
# subset dataframe 

data.SNP_QC <-subset(data.SNP_QC, array ==2)

# autosomal variants in both arrays = 733,322 
nrow(data.SNP_QC)

[1] 733322

In [ ]:
# create list.notypedinbotharrays for reference
  
list.notypedinbotharrays <-subset(data.SNP_QC, ! array ==2, select=c(rs_id))
outfile.name2 <- "list.variantsnotypedinbotharrays.txt"
write.table(list.notypedinbotharrays, outfile.name2, quote=FALSE, col.names=TRUE, row.names=FALSE, sep="\t")

## Step 4. Select variants passing all batch QC tests

In [42]:
both_arrays <- data.SNP_QC %>%
     filter_at(vars(starts_with("Batch")), all_vars(. > 0)) %>%
     filter_at(vars(starts_with("UKBiLEVEAX")), all_vars(. > 0)) %>%
     filter(array==2)
    
#create sumbatches variable (sums batch-specific indicators for each column, sumbatches should equal 106 for inclusion)        
data.SNP_QC$sumbatches <- rowSums(data.SNP_QC[,10:115])

# subset dataframe 
data.SNP_QC <-subset(data.SNP_QC,sumbatches  ==106, select=c(rs_id))

# autosomal variants = 687,004
nrow(data.SNP_QC)
write.table(data.SNP_QC, "SNPs_autosomalvariantspassingbatchqc_120219.txt", quote=FALSE, col.names=TRUE, row.names=FALSE, sep="\t")

[1] 687004

In [ ]:
# create list.failbatchQCtests for reference
list.failbatchQCtests <-subset(data.SNP_QC, ! sumbatches ==106, select=c(rs_id))
outfile.name3 <- "list.variantsfailingoneormorebatchQCtests.txt"
write.table(list.failbatchQCtests, outfile.name3, quote=FALSE, col.names=TRUE, row.names=FALSE, sep="\t")

## Step 5. select SNPs only within PLINK  (--SNPS-ONLY flag)

In [2]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed
# Original bfile containing all of the samples Columbias's cluster
genoFile=$UKBB_yale/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.bed
#To keep the variants that passed previous qc: autosomal, both batches, pass qc within batches
keep_variants=~/UKBiobank/genotype_files_processed/SNPs_autosomalvariantspassingbatchqc_120219.txt
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/snponly_originalbed_$(date +"%Y-%m-%d").sbatch
snps_only=True
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'
name='082421'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --snps_only $snps_only
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

/home/dmc2245/miniconda3/lib/python3.8/site-packages/sos_notebook/kernel.py:1334: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  msg['msg_id'] = self._parent_header['header']['msg_id']


INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/snponly_originalbed_2021-08-24.sbatch
INFO: Workflow csg (ID=wc0e398d163b9a337) is executed successfully with 1 completed step.



In [ ]:
## Step 6. Run the PERL SCRIPT for Missingness, MAF, and HWE
  # Note that this script will yield two output files: <nameoffile>.QC and <nameoffile>.out
  # Use <nameoffile>.QC to filter out the variants
  # Keep <nameoffile>.out as reference

  data.perl_SNP_QC <- read.table("/research_storage/dbgap/work/Yasmmyn/UK_Biobank/geneQC/try2_UKB_SNPs.QC", header=TRUE, sep="\t")

  # Step 6a. missingness
  names(data.perl_SNP_QC)[1] <-"rs_id"
  list.exclude.perl_SNP_QC1  <- subset(data.perl_SNP_QC,data.perl_SNP_QC$MISS==1,select=c(rs_id))
  list.exclude.perl_SNP_QC1  <- as.matrix(list.exclude.perl_SNP_QC1)


  data.SNP_QC <-subset(data.SNP_QC, ! rs_id %in% list.exclude.perl_SNP_QC1)

  # SNPs = 652,399
  nrow(data.SNP_QC)

  # Step 6b. HWE
  
  list.exclude.perl_SNP_QC2 <- subset(data.perl_SNP_QC,data.perl_SNP_QC$HWE==1,select=c(rs_id))
  list.exclude.perl_SNP_QC2 <- as.matrix(list.exclude.perl_SNP_QC2)

  data.SNP_QC <-subset(data.SNP_QC, ! rs_id %in% list.exclude.perl_SNP_QC2)

  # SNPs = 622,266
  nrow(data.SNP_QC)
  
  # Step 6c. MAF 
  
  list.exclude.perl_SNP_QC3  <- subset(data.perl_SNP_QC,data.perl_SNP_QC$MAF==1,select=c(rs_id))
  list.exclude.perl_SNP_QC3  <- as.matrix(list.exclude.perl_SNP_QC3)

  data.SNP_QC <-subset(data.SNP_QC, ! rs_id %in% list.exclude.perl_SNP_QC3)

  
  # SNPs = 541,312
  nrow(data.SNP_QC)

  # note that these numbers are off by 12,515 because I deleted indels using the --SNPS-ONLY flag in PLINK

# Sample QC summary

Original file downloaded from the UKBB 

~/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.fam

- Starting number of variants: 784,256 (starting with only autosomal variants)
- Starting number of individuals: 488,377 (individuals that have both genotype and phenotype information)

* Genetic and reported sex match --> 487,849
* Sex chromosomes non XX - XY --> 487,379
* Outliers heterozygosity/missing rate --> 486,416
* Individuals call rate > 99% --> 436,698

The R script and submission file are present here

`~/UKBiobank/genotype_files_processed/082421_sample_qc.R`

`~/UKBiobank/genotype_files_processed/082421_sample_qc.sh`

## Step 1. Import list of people with genotype data into a dataframe

In [50]:
data.havegenotypes <- read.table("~/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.fam", header= FALSE, stringsAsFactors = FALSE)
names(data.havegenotypes) <-c("FID","IID","ignore1", "ignore2", "ignore3", "ignore4")
cat("The number of individuals with genotype data is:",nrow(data.havegenotypes),"\n")
 # n = 488377 subjects

The number of individuals with genotype data is: 488377 


## Step 2. Import phenotype data and create a dataframe

In [51]:
library(data.table)
dat <- fread("~/UKBiobank/data/ukbb_databases/ukb47922_updatedAug2021/ukb47922.tab", header=TRUE, sep="\t", select = c("f.eid","f.31.0.0","f.22001.0.0","f.22019.0.0","f.22027.0.0","f.22021.0.0","f.22006.0.0"))

In [52]:
head(dat)

f.eid,f.31.0.0,f.22001.0.0,f.22019.0.0,f.22027.0.0,f.22021.0.0,f.22006.0.0
<int>,<int>,<int>,<int>,<int>,<int>,<int>
1000019,0,0,NA,NA,1,1
1000022,1,1,NA,NA,0,1
1000035,1,1,NA,NA,1,1
1000046,0,0,NA,NA,1,1
1000054,0,0,NA,NA,1,1
1000063,1,1,NA,NA,0,1


In [53]:
#mydata <- read.table("~/UKBiobank/data/ukbb_databases/ukb47922_updatedAug2021/ukb47922.tab", header=TRUE, sep="\t")
names(dat)[1] <- "IID" 
cat("The number of individuals with phenotype data is:",nrow(dat),"\n")
 # n = 502,461 subjects

The number of individuals with phenotype data is: 502461 


## Step 3. Merge the dataframes 

In [54]:
data.haveboth <-merge(data.havegenotypes, dat, by="IID", all=FALSE)
cat("The number of individuals with both phenotype and genotype data:",nrow(data.haveboth),"\n")
# n = 487,849(had phenotype data, #528 indiduals removed)

The number of individuals with both phenotype and genotype data: 488221 


## Step 4. Additional sex checks  

In [55]:
#4a. genetic sex vs. self-reported sex

data.haveboth$sex_match<- (data.haveboth$f.22001.0.0 == data.haveboth$f.31.0.0)
sexnomatch <- subset(data.haveboth, data.haveboth$sex_match=="FALSE")
data.haveboth<- subset(data.haveboth, data.haveboth$sex_match=="TRUE")
cat("The number of individuals whose reported and genetic sex match is:",nrow(data.haveboth),"\n")
cat("The number of individuals whose reported and genetic sex does NOT match is:",nrow(sexnomatch),"\n")
# n = 487,379 subjects match for sex

The number of individuals whose reported and genetic sex match is: 487849 
The number of individuals whose reported and genetic sex does NOT match is: 372 


In [56]:
#4b. Identify subjects with sex chromosome karyotypes putatively different from XX or XY

aneu.toexclude  <- subset(data.haveboth, data.haveboth$f.22019.0.0==1, select=c(IID))
aneu.toexclude  <- as.matrix(aneu.toexclude)

data.haveboth <-subset(data.haveboth, ! IID %in% aneu.toexclude)
cat("The number of individuals with aneuploidies is:",nrow(aneu.toexclude),"\n")
cat("The number of individuals with XX and XY",nrow(data.haveboth),"\n")
#n = 486,416 subjects remain

The number of individuals with aneuploidies is: 470 
The number of individuals with XX and XY 487379 


## Step 5. Identify subjects that are outliers in heterozygosity and missing rates

In [57]:
list.toexclude <- subset(data.haveboth, data.haveboth$f.22027.0.0==1, select=c(IID))
list.toexclude  <- as.matrix(list.toexclude)

data.haveboth <-subset(data.haveboth, ! IID %in% list.toexclude)
cat("The number of individuals that are outliers for heterozygosity or missing rates is:",nrow(list.toexclude),"\n")
cat("The number of individuals that are not outliers for heterozygosity or missing rates is:",nrow(data.haveboth),"\n")

The number of individuals that are outliers for heterozygosity or missing rates is: 963 
The number of individuals that are not outliers for heterozygosity or missing rates is: 486416 


In [71]:
# Save the IID that pass these QC filters to a file
ID_keep <- data.haveboth[,1, drop=FALSE]
#write.table(ID_keep,'~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_keep.txt', row.names = FALSE, col.names = FALSE)

## Step 6. Filter using plink for individual call rate > 99%

After selecting the individuals and applying mind=0.01 (keep individuals with call rate > 99%) 

49718 samples removed due to missing genotype data (--mind)

In [6]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed/082621_sampleqc
# bfile with variant_qc_1
genoFile=~/UKBiobank/genotype_files_processed/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.bed
#To keep the samples that passed sample_qc_1
keep_samples=~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_keep.txt
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/sampleqc_variantfilteredbed_$(date +"%Y-%m-%d").sbatch
maf_filter=0
geno_filter=0
hwe_filter=0
# Keep individuals with call rate > 99%
mind_filter=0.01
mem='30G'
name='082621'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/sampleqc_variantfilteredbed_2021-08-26.sbatch
INFO: Workflow csg (ID=wb3a0e834b911693a) is executed successfully with 1 completed step.



## Use a call rate >90%

In [92]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed/082621_sampleqc_call90
# bfile with variant_qc_1
genoFile=~/UKBiobank/genotype_files_processed/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.bed
#To keep the samples that passed sample_qc_1
keep_samples=~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_keep.txt
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/sampleqc_call90_$(date +"%Y-%m-%d").sbatch
maf_filter=0
geno_filter=0
hwe_filter=0
# Keep individuals with call rate > 90%
mind_filter=0.1
mem='30G'
name='082621'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/sampleqc_call90_2021-08-31.sbatch
INFO: Workflow csg (ID=w5c2d59f208020dfc) is executed successfully with 1 completed step.



## Step 7. Relatedness using variable f.22021.0.0

In [70]:
### Read in the removed individuals after the sample missingness filter (mind=0.01) 99% call rate
removed_IID <- read.table("~/UKBiobank/genotype_files_processed/082621_sampleqc/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.082621_sampleqc.filtered.mindrem.id", header=FALSE)
names(removed_IID) <-c("FID","IID")

In [59]:
### Read in the removed individuals after the sample missingness filter (mind=0.1) 90% call rate
retained_IID <- read.table("~/UKBiobank/genotype_files_processed/082621_sampleqc_call90/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.082621_sampleqc_call90.filtered.fam", header=FALSE)
names(retained_IID) <-c("FID","IID","ignore1", "ignore2", "ignore3", "ignore4")

In [84]:
data.related <- subset(data.haveboth, data.haveboth$f.22021.0.0==1)  #at least one relative identified
ID_related <- data.related %>%
    filter(!IID %in% removed_IID$IID) %>%
    select ("FID", "IID")
cat("The number of individuals that are related in the sample is:",nrow(data.related),"\n")
cat("The number of individuals that are related in the sample after filtering for sample missingess is:",nrow(ID_related),"\n")
write.table(ID_related, '~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_related.txt', sep="\t", row.names = FALSE, col.names= FALSE)

The number of individuals that are related in the sample is: 147252 
The number of individuals that are related in the sample after filtering for sample missingess is: 132222 


In [13]:
data.related <- subset(data.haveboth, data.haveboth$f.22021.0.0==1)  #at least one relative identified
ID_related <- data.related %>%
    filter(!IID %in% retained_IID$IID) %>%
    select ("FID", "IID")
cat("The number of individuals that are related in the sample is:",nrow(data.related),"\n")
cat("The number of individuals that are related in the sample after filtering for sample missingess is:",nrow(ID_related),"\n")
#write.table(ID_related, '~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_related.txt', sep="\t", row.names = FALSE, col.names= FALSE)

The number of individuals that are related in the sample is: 147252 
The number of individuals that are related in the sample after filtering for sample missingess is: 0 


### Determine kinship using king

In [11]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
##Columbia's variables
cwd=$UKBB_PATH/genotype_files_processed/082621_king
genoFile=$UKBB_PATH/genotype_files_processed/082621_sampleqc/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.082621_sampleqc.filtered.bed
king_sbatch=$USER_PATH/UKBB_GWAS_dev/output/king_genoarray_$(date +"%Y-%m-%d").sbatch
kinship=0.0625
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
numThreads=20
mem='30G'
walltime='60h'

king_args="""king
    --cwd $cwd
    --genoFile $genoFile
    --kinship $kinship
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
    --walltime $walltime
    --no-maximize-unrelated
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $king_sbatch \
    --args "$king_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/king_genoarray_2021-08-27.sbatch
INFO: Workflow csg (ID=w9ab1d59b1ee0d16a) is executed successfully with 1 completed step.



## Step 8 Variant QC number 2

### Get the unrelated individuals only, do quality controls for variant missingness (geno=0.01), HWE (5e-08) and maf filter (0.01)

In [85]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed/083021_sampleqc2
# bfile with variant_qc_1
genoFile=~/UKBiobank/genotype_files_processed/082621_sampleqc/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.082621_sampleqc.filtered.bed
#To remove related samples
remove_samples=~/UKBiobank/genotype_files_processed/082421_sampleQC_IID_related.txt
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/sampleqc2_$(date +"%Y-%m-%d").sbatch
maf_filter=0.01
geno_filter=0.01
hwe_filter=5e-08
# Set mind filter to 0 not to filter out more individuals based on sample missingness
mind_filter=0
mem='30G'
name='083021'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --remove_samples $remove_samples
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/sampleqc2_2021-08-31.sbatch
INFO: Workflow csg (ID=w039a30c991021747) is executed successfully with 1 completed step.



## Create the final file for analyses call rate >99%

In [86]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed/083021_sample_variant_qc_final
# orginal bfile
genoFile=$UKBB_yale/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.bed
#To keep samples
keep_samples=~/UKBiobank/genotype_files_processed/082621_sampleqc/cache/UKB_genotypedatadownloaded083019.082621_sampleqc.qc_pass.id
# To keep variants
keep_variants=~/UKBiobank/genotype_files_processed/083021_sampleqc2/cache/UKB_genotypedatadownloaded083019.082621_sampleqc.083021_sampleqc2.qc_pass.snplist
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/sampleqc_final_$(date +"%Y-%m-%d").sbatch
## All filters set to 0 
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'
name='083021'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/sampleqc2_2021-08-31.sbatch
INFO: Workflow csg (ID=wb46da603845062d4) is executed successfully with 1 completed step.



## Create the final file for analyses call rate >90% (this is the one used downstream for PCA and LMM analysis)

In [77]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/genotype_files_processed/090221_sample_variant_qc_final_callrate90
# orginal bfile
genoFile=$UKBB_yale/pleiotropy_geneticfiles/UKB_originalgenotypefilesdownloaded083019/UKB_genotypedatadownloaded083019.bed
#To keep samples
keep_samples=~/UKBiobank/genotype_files_processed/082621_sampleqc_call90/cache/UKB_genotypedatadownloaded083019.genotype_files_processed.filtered.extracted.082621_sampleqc_call90.filtered.qc_pass.id
# To keep variants
keep_variants=~/UKBiobank/genotype_files_processed/083021_sampleqc2/cache/UKB_genotypedatadownloaded083019.082621_sampleqc.083021_sampleqc2.qc_pass.snplist
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/sampleqc_final_$(date +"%Y-%m-%d").sbatch
## All filters set to 0 
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0
mem='30G'
name='090221'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --keep_samples $keep_samples
    --keep_variants $keep_variants
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/sampleqc_final_2021-09-02.sbatch
INFO: Workflow csg (ID=wd3cc554baec8ffe9) is executed successfully with 1 completed step.



### Ancestry restriction (Megan pipeline to use only people that self-identify as white-Bristish)

In this case we are defining our own ancestry based on the PC analysis

In [ ]:
data.haveboth <- subset(data.haveboth, data.haveboth$f.22006.0.0==1)
#n = 408245

## Callrate

In [ ]:
data.imissing <-read.table("/research_storage/scratch/UKBiobank/genotype_files/my_SNP_QC/bfiles_created_along_the_way/UKB_autosomalvariants_passing_batchQC_noindels_unrelated_whiteBritishsubjects_passing_standardexclusions_samplecallrt.irem", header=FALSE, stringsAsFactors = FALSE)

## Determine how many of the individuals from the exome data are removed with the sampleQC in the genotype array data

In [58]:
## Read-in 'white' individuals 

exomed_IID <- read.table('/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind')
names(exomed_IID) <-c("FID","IID")

In [75]:
head(exomed_IID)
nrow(exomed_IID)

,FID,IID
,<int>,<int>
1,1000019,1000019
2,1000035,1000035
3,1000078,1000078
4,1000081,1000081
5,1000198,1000198
6,1000210,1000210


[1] 189010

In [60]:
ID_exome_not_geno <- exomed_IID %>%
    filter(!IID %in% retained_IID$IID) %>%
    select ("FID", "IID")

In [61]:
# Individuals present in the exome data but that did not pass sample-QC on the genotype array data
nrow(ID_exome_not_geno)

[1] 536

In [68]:
head(sexnomatch)

,IID,FID,ignore1,ignore2,ignore3,ignore4,f.31.0.0,f.22001.0.0,f.22019.0.0,f.22027.0.0,f.22021.0.0,f.22006.0.0,sex_match
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<lgl>
374,1003854,1003854,0,0,1,-9,0,1,NA,NA,0,1,FALSE
621,1006361,1006361,0,0,1,-9,0,1,NA,NA,0,1,FALSE
1158,1011973,1011973,0,0,1,-9,0,1,NA,NA,0,1,FALSE
1297,1013439,1013439,0,0,1,-9,0,1,NA,NA,1,1,FALSE
1939,1019998,1019998,0,0,2,-9,1,0,1,NA,1,1,FALSE
3881,1040038,1040038,0,0,2,-9,1,0,1,NA,1,1,FALSE


In [62]:
# How many removed because sex did not match
nomatchsex <-  ID_exome_not_geno %>%
    filter(IID %in% sexnomatch$IID)
nrow(nomatchsex)

[1] 0

In [63]:
head(aneu.toexclude)
aneu.toexclude <- as.data.frame(aneu.toexclude)

,IID
93,1000971
1780,1018401
1849,1019099
2368,1024453
2630,1027190
4269,1044050


In [64]:
# How many have sex aneuploidies
aneu <- ID_exome_not_geno %>%
    filter(IID %in% aneu.toexclude$IID)
nrow(aneu)

[1] 162

In [65]:
head(list.toexclude)
list.toexclude <- as.data.frame(list.toexclude)

,IID
1396,1014455
1644,1017024
1699,1017579
2996,1030922
3350,1034540
3795,1039146


In [67]:
# How many removed for being outliers in heterozygosity and missing rates
outliers <- ID_exome_not_geno %>%
    filter(IID %in% list.toexclude$IID)
nrow(outliers)

[1] 365

In [ ]:
nrow(ID_exome_not_geno %>% filter(IID %in% dat$IID))

## Determine sample missingness for merged exomes 

In [3]:
# This is the path to the data transferred from Yale
UKBB_yale=/mnt/mfs/statgen/archive/UKBiobank_Yale_transfer
UKBB_PATH=/mnt/mfs/statgen/UKBiobank
USER_PATH=$HOME/project
OUT_PATH=$USER_PATH/UKBB_GWAS_dev/output
tpl_file=$USER_PATH/bioworkflows/admin/csg.yml
container_lmm=$HOME/containers/lmm.sif
#Columbia's cluster
cwd=~/UKBiobank/data/exome_files/project_VCF/072721_run/merged_plink/mind_0.1
# orginal bfile
genoFile=~/UKBiobank/data/exome_files/project_VCF/072721_run/merged_plink/ukb23155_qc_merged.bed
gwasqc_sos=$USER_PATH/bioworkflows/GWAS/GWAS_QC.ipynb
gwasqc_sbatch=$USER_PATH/UKBB_GWAS_dev/output/exome_sample_missingess_$(date +"%Y-%m-%d").sbatch
##remove individuals with > 10% genotypes missing
maf_filter=0
geno_filter=0
hwe_filter=0
mind_filter=0.1
mem='30G'
name='090221'
job_size=1
numThreads=2

gwasqc1_args="""qc:1
    --cwd $cwd
    --genoFile $genoFile
    --maf_filter $maf_filter
    --geno_filter $geno_filter
    --hwe_filter $hwe_filter
    --mind_filter $mind_filter
    --numThreads $numThreads 
    --job_size $job_size
    --container_lmm $container_lmm
    --mem $mem
"""

sos run ~/project/UKBB_GWAS_dev/admin/Get_Job_Script.ipynb csg \
    --template-file $tpl_file \
    --workflow-file $gwasqc_sos \
    --to-script $gwasqc_sbatch \
    --args "$gwasqc1_args"

INFO: Running csg: Configuration for Columbia csg partition cluster
INFO: csg is completed.
INFO: csg output:   /home/dmc2245/project/UKBB_GWAS_dev/output/exome_sample_missingess_2021-09-03.sbatch
INFO: Workflow csg (ID=wfadfd852309f5295) is executed successfully with 1 completed step.



In [6]:
## Select individuals for PCA phenofile 
white <- read.table("~/UKBiobank_Yale_transfer/pleiotropy_R01/ukb43978_OCT2020/dc2325_phenotypes/030821_ukb42495_exomed_white_189010ind.pheno", sep="\t", header=TRUE)

In [7]:
head(white)

,FID,IID,ethnicity
,<int>,<int>,<fct>
1,1000019,1000019,British
2,1000035,1000035,British
3,1000078,1000078,British
4,1000081,1000081,British
5,1000198,1000198,British
6,1000210,1000210,British


In [17]:
pca_keep <- read.table("/mnt/mfs/statgen/UKBiobank/results/083021_PCA_results/europeans/cache/UKB_genotypedatadownloaded083019.090221_sample_variant_qc_final_callrate90.filtered.extracted.europeans.filtered.fam")
names(pca_keep) <-c("FID","IID", "ignore1","ignore2", "ignore3", "ignore4")

In [18]:
head(pca_keep)

,FID,IID,ignore1,ignore2,ignore3,ignore4
,<int>,<int>,<int>,<int>,<int>,<int>
1,1000019,1000019,0,0,2,-9
2,1000035,1000035,0,0,1,-9
3,1000078,1000078,0,0,2,-9
4,1000081,1000081,0,0,1,-9
5,1000198,1000198,0,0,2,-9
6,1000210,1000210,0,0,1,-9


In [20]:
library(tidyverse)
selected <- white %>%
    filter(IID %in% pca_keep$IID)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [21]:
nrow(selected)

[1] 188474